# Portugal

In [83]:
from json import JSONDecodeError
import pandas as pd
import geopandas as gpd
from bs4 import BeautifulSoup
import numpy as np
import requests

pd.options.mode.chained_assignment = None

## Fetch results

In [84]:
base_url = 'https://www.eleicoes.mai.gov.pt'

In [85]:
def get_data(id):
    url = f'{base_url}/europeias2014/static-data/territory-results/TERRITORY-RESULTS-LOCAL-{id}-EUR.json'
    r = requests.get(url)
    try:
        data = r.json().get('currentResults')
    except JSONDecodeError:
        data = {}
    return {
        'voters': data.get('numberVoters'),
        'eligible': data.get('subscribedVoters'),
        'turnout': data.get('percentageVoters')
    }

In [86]:
%%time
data = []
for i, row in geo.iterrows():
    data.append(get_data(row.Dicofre))

CPU times: user 1min 4s, sys: 3.61 s, total: 1min 8s
Wall time: 17min 54s


In [89]:
df = pd.DataFrame(data)

In [91]:
df.to_csv('../../data/raw/prt.csv', index=False)

## Merge

In [92]:
geo = gpd.read_file('../../data/shapefiles/prt/Cont_AAD_CAOP2017.shp')

In [93]:
geo['turnout'] = [x['turnout'] for x in data]
geo['voters'] = [x['voters'] for x in data]
geo['eligible'] = [x['eligible'] for x in data]

## Export

In [95]:
geo = geo.to_crs(epsg=4326)

In [97]:
geo = geo[['Freguesia', 'turnout', 'geometry']]
geo.columns = ['name', 'turnout', 'geometry']
geo['country'] = 'prt'

In [98]:
geo.to_file('../../data/processed/prt')